In [1]:
import os
import time
from datetime import datetime
from pathlib import Path

import akshare as ak
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [2]:
fund_code = '510300'
init_invest = 70000
cash = 30000
buy_point, sell_point = 0.2, 0.8

In [3]:
def is_gt_24hours(early_dt, later_dt):
    return (later_dt - early_dt).days >= 1

In [4]:
def cache_fund_to_csv(fund_code, filename):
    open_fund_df = ak.fund_open_fund_info_em(fund=fund_code, indicator="单位净值走势")
    open_fund_df.to_csv(filename, index_label='index')

In [5]:
def recent_10_years(df):
    return df.tail(10 * 12)

In [6]:
fund_cache_file = f'../test-cached/open_fund/{fund_code}.csv'

if not Path(fund_cache_file).exists():
    cache_fund_to_csv(fund_code, fund_cache_file)

fund_dt = datetime.fromtimestamp(os.stat(fund_cache_file).st_mtime)

In [7]:
if is_gt_24hours(fund_dt, datetime.now()):
    cache_fund_to_csv(fund_code, fund_cache_file)

In [8]:
fund_df = pd.read_csv(fund_cache_file, parse_dates=['净值日期'], index_col=['净值日期'])
fund_df.drop(columns=['index', '日增长率'], inplace=True)
by_month_df = fund_df.resample("MS").first()
# by_month_df

In [9]:
recent_10_years_df = recent_10_years(by_month_df)
recent_10_years_df.reset_index(inplace=True)

first_shares = round(init_invest / recent_10_years_df.iloc[0]['单位净值'], 4)

recent_10_years_df['持有份额'] = first_shares
recent_10_years_df['总资产'] = round(
    recent_10_years_df['单位净值'] * recent_10_years_df['持有份额'] + cash, 2
)

# recent_10_years_df

/var/folders/sn/qsm415tn5157jpntkj3qhhgr0000gn/T/ipykernel_10367/991492369.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_10_years_df['持有份额'] = first_shares
/var/folders/sn/qsm415tn5157jpntkj3qhhgr0000gn/T/ipykernel_10367/991492369.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_10_years_df['总资产'] = round(


In [10]:
recent_10_years_df['最近一年增长'] = recent_10_years_df['单位净值'].pct_change(periods=12)

# 处理第一年的数据
recent_10_years_df['最近一年增长'].iloc[0] = 0
for i in range(12):
    recent_10_years_df['最近一年增长'].iloc[i + 1] = (
        recent_10_years_df['总资产'].iloc[i + 1] / recent_10_years_df['总资产'].iloc[0]
    ) - 1

recent_10_years_df.to_csv(
    f'../test-cached/open_fund/{fund_code}_滚动收益.csv', index_label='index'
)
recent_10_years_df

/var/folders/sn/qsm415tn5157jpntkj3qhhgr0000gn/T/ipykernel_10367/2557013430.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_10_years_df['最近一年增长'] = recent_10_years_df['单位净值'].pct_change(periods=12)
/var/folders/sn/qsm415tn5157jpntkj3qhhgr0000gn/T/ipykernel_10367/2557013430.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_10_years_df['最近一年增长'].iloc[0] = 0
/var/folders/sn/qsm415tn5157jpntkj3qhhgr0000gn/T/ipykernel_10367/2557013430.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

,净值日期,单位净值,持有份额,总资产,最近一年增长
0,2012-09-01,2.2650,30905.0773,100000.00,0.000000
1,2012-10-01,2.3060,30905.0773,101267.11,0.012671
2,2012-11-01,2.3340,30905.0773,102132.45,0.021324
3,2012-12-01,2.1420,30905.0773,96198.68,-0.038013
4,2013-01-01,2.5270,30905.0773,108097.13,0.080971
...,...,...,...,...,...
115,2022-04-01,4.2728,30905.0773,162051.21,-0.164114
116,2022-05-01,4.0087,30905.0773,153889.18,-0.209002
117,2022-06-01,4.0891,30905.0773,156373.95,-0.235982
118,2022-07-01,4.4975,30905.0773,168995.59,-0.146503


In [11]:
quantiles = recent_10_years_df['最近一年增长'].quantile([buy_point, 0.5, sell_point])
quantiles

0.2   -0.123994
0.5    0.067611
0.8    0.235196
Name: 最近一年增长, dtype: float64